In [12]:
import numpy as np
import pandas as pd

np.random.seed(42)

n_A = 10000
n_B = 10000

# Группа A (контроль)
group_A = pd.DataFrame({
    "user_id": np.arange(1, n_A + 1),
    "group": "A",
    "age": np.random.randint(18, 60, n_A),                        # Возраст
    "gender": np.random.choice(["male", "female"], n_A),          # Пол
    "device": np.random.choice(["desktop", "mobile"], n_A),       # Устройство
    "income": np.random.normal(50000, 12000, n_A).astype(int),    # Доход
    "conversion": np.random.binomial(1, 0.10, n_A)                # 10% конверсия
})

# Группа B (тест)
group_B = pd.DataFrame({
    "user_id": np.arange(n_A + 1, n_A + n_B + 1),
    "group": "B",
    "age": np.random.randint(18, 60, n_B),
    "gender": np.random.choice(["male", "female"], n_B),
    "device": np.random.choice(["desktop", "mobile"], n_B),
    "income": np.random.normal(50000, 12000, n_B).astype(int),    # Чуть выше доход в тестовой
    "conversion": np.random.binomial(1, 0.12, n_B)                # 13% конверсия (улучшение)
})

ab_data = pd.concat([group_A, group_B], ignore_index=True)

ab_data.head()

,user_id,group,age,gender,device,income,conversion
0,1,A,56,male,desktop,48211,0
1,2,A,46,female,mobile,34820,0
2,3,A,32,male,mobile,30573,0
3,4,A,25,male,desktop,62269,0
4,5,A,38,female,mobile,36831,0


In [13]:
from scipy.stats import ttest_ind

# Получим выборки по группам
A = ab_data[ab_data['group'] == 'A']['conversion']
B = ab_data[ab_data['group'] == 'B']['conversion']

# Применим t-тест для независимых выборок
t_stat, p_value = ttest_ind(A, B)

print(f"t-статистика: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")

t-статистика: -2.7832
p-value: 0.0054


In [14]:
from scipy.stats import ttest_ind, chi2_contingency

# 1. Числовые ковариты: 'age', 'income'
print("Проверка однородности по числовым коваритам")
for col in ['age', 'income']:
    A = ab_data[ab_data['group'] == 'A'][col]
    B = ab_data[ab_data['group'] == 'B'][col]
    t_stat, p_val = ttest_ind(A, B)
    print(f"{col}: t={t_stat:.3f}, p-value={p_val:.4f}")

# 2. Категориальные ковариты: 'gender', 'device'
print("\nПроверка однородности по категориальным коваритам")
for col in ['gender', 'device']:
    contingency_table = pd.crosstab(ab_data['group'], ab_data[col])
    chi2, p_val, _, _ = chi2_contingency(contingency_table)
    print(f"{col}: chi2={chi2:.3f}, p-value={p_val:.4f}")

Проверка однородности по числовым коваритам
age: t=0.197, p-value=0.8441
income: t=0.186, p-value=0.8527

Проверка однородности по категориальным коваритам
gender: chi2=0.051, p-value=0.8210
device: chi2=0.001, p-value=0.9774
